In [1]:
!pip install torch-scatter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for torch-scatter: filename=torch_scatter-2.1.2-cp310-cp310-linux_x86_64.whl size=504020 sha256=6d28adeb1905e9146ce5ee3558f40002c07427b89988c58de9852cd8d1bd1d5d
  Stored in directory: /root/.cache/pip/wheels/92/f1/2b/3b46d54b134259f58c8363568569053248040859b1a145b3ce
Successfully built torch-scatter


In [2]:
!pip install torch_geometric



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.6 MB/s eta 0:00:00


In [3]:
import os
import torch
import random
import pandas as pd
import numpy as np

import torch
import torch.nn.functional as F
import torch_geometric
from torch_geometric.nn import GCNConv
from torch_geometric.data import Data
from torch_geometric.data import DataLoader
from collections import defaultdict

In [ ]:

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')#

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def adjacency_to_edge_index(adjacency_matrix):
    edge_index = []
    for i in range(adjacency_matrix.shape[0]):
        for j in range(adjacency_matrix.shape[1]):
            if adjacency_matrix[i, j] != 0:
                edge_index.append([i, j])
    return torch.tensor(edge_index, dtype=torch.long).t()


def load_graph_and_label(graph_file, label_file):
    # Read the features
    features_df = pd.read_csv(graph_file, delimiter=',', header=None, skiprows=1, nrows=3)

    features_df = features_df.apply(pd.to_numeric, errors='coerce')
    if features_df.isnull().values.any():
        features_df = features_df.fillna(0)

    feature_tensor = torch.tensor(features_df.values, dtype=torch.float)
    adjacency_matrix = pd.read_csv(graph_file, delimiter=',', header=None, skiprows=5)
    adjacency_matrix_tensor = torch.tensor(adjacency_matrix.values, dtype=torch.long)

    edge_index = adjacency_to_edge_index(adjacency_matrix_tensor)


    #print(edge_index.shape)
    label = pd.read_csv(label_file, header=None).values[0,0]
    data = Data(x=feature_tensor, edge_index=edge_index, y=torch.tensor(label, dtype=torch.long))

    return data


In [ ]:
label_mappings = {
    0: [0, 1, 2],
    1: [0, 2, 1],
    2: [1, 0, 2],
    3: [1, 2, 0],
    4: [2, 0, 1],
    5: [2, 1, 0]
}


In [ ]:
"""
gnn_graphs_dir = '/content/drive/MyDrive/GNN_Colab/sampled_gnn_data-Copy/gnn_graphs'
gnn_labels_dir = "/content/drive/MyDrive/GNN_Colab/sampled_gnn_data-Copy/gnn_labels"

dataset = []
i=0
for filename in os.listdir(gnn_graphs_dir):
      if filename.endswith(".csv"):
          graph_file = os.path.join(gnn_graphs_dir, filename)
          label_file = os.path.join(gnn_labels_dir, filename.replace('_combined.csv', '_label.txt'))
          #/content/drive/MyDrive/GNN_Colab/gnn_data/gnn_graphs
          if os.path.exists(graph_file) and os.path.exists(label_file):
              data = load_graph_and_label(graph_file, label_file)
              dataset.append(data)
          else:
              print(f"Files not found for: {filename} ",graph_file,label_file)

dataset_file_path = '/content/drive/MyDrive/GNN_Colab/final_full_dataset_file.pt'  # Specify the path to save your dataset

torch.save(dataset, dataset_file_path)
"""

In [ ]:
dataset = torch.load('/content/drive/MyDrive/GNN_Colab/final_full_dataset')

In [ ]:
print((dataset[0]))

Data(x=[3, 14], edge_index=[2, 6], y=4, name='poly1-perm0.txt.ml')


In [ ]:
random.shuffle(dataset)

dataset_groups = defaultdict(list)

for data in dataset:
    if data.edge_index.dim() == 2:
        num_edges = data.edge_index.size(1)
    else:
        num_edges = 0

    dataset_groups[num_edges].append(data)

for num_edges in dataset_groups:
    random.shuffle(dataset_groups[num_edges])


In [ ]:
def dataset_to_dataframe(dataset):
    data_list = []
    for data in dataset:
        x_flattened = data.x.reshape(-1).numpy()
        data_dict = {'features': x_flattened}
        if hasattr(data, 'y'):
            data_dict['label'] = data.y.item()
        if hasattr(data, 'name'):
            data_dict['name'] = data.name
        data_list.append(data_dict)

    df = pd.DataFrame(data_list)

    return df

df = dataset_to_dataframe(dataset)

features_df = pd.DataFrame(df['features'].tolist(), index=df.index)
df_expanded = pd.concat([df.drop(columns=['features']), features_df], axis=1)

print(df.head())

output_file_path = '/content/drive/MyDrive/CAD_Project/extra_features_data'
df_expanded.to_csv(output_file_path, index=False)

KeyboardInterrupt: 

In [ ]:
print(df_expanded.shape)

(41369, 44)


In [ ]:
print(dataset[0])

for graph_data in dataset[0:10]:
        graph_label = graph_data.y
        print(graph_label)

Data(x=[3, 14], edge_index=[2, 2], y=4, name='poly864-perm4.txt.ml')
tensor(4)
tensor(1)
tensor(4)
tensor(0)
tensor(5)
tensor(0)
tensor(5)
tensor(2)
tensor(1)
tensor(3)


In [ ]:
def transform_labels(dataset, label_mappings):
    node_labels = []
    for graph_data in dataset:
        if isinstance(graph_data, Data):
            graph_label = graph_data.y.item()
            node_order = label_mappings[graph_label]
            node_labels.append([node_order[node_idx] for node_idx in range(3)])
    return node_labels

RuntimeError: a Tensor with 3 elements cannot be converted to Scalar

In [ ]:
print(len(dataset))

41369


In [ ]:
for data in dataset:
    if hasattr(data, 'name'):
        del data.name


In [ ]:
def update_dataset_labels(dataset, label_mappings):
    node_labels = transform_labels(dataset, label_mappings)
    if len(node_labels) != len(dataset):
        print("Error: The number of transformed labels does not match the number of graphs in the dataset.")
        return
    for data, new_labels in zip(dataset, node_labels):
        data.y = torch.tensor(new_labels, dtype=torch.long)

update_dataset_labels(dataset, label_mappings)

for graph_data in dataset[:10]:
    print(graph_data.y)


RuntimeError: a Tensor with 3 elements cannot be converted to Scalar

In [ ]:
print(len(dataset))

41369


In [ ]:
subset_size = int(len(dataset))
subset_dataset = dataset[:subset_size]

train_size = int(len(subset_dataset) * 0.8)
train_dataset = subset_dataset[:train_size]
test_dataset = subset_dataset[train_size:]

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

data_loader = DataLoader(dataset, batch_size=32, shuffle=False)

Training batches: 1035
Testing batches: 259
Total dataset batches: 1293


In [ ]:
import copy

dataset_modified = copy.deepcopy(dataset)

for data in dataset_modified:
    data.x = torch.cat((data.x[:, :4], data.x[:, 6:-2]), dim=1)



In [ ]:
print(dataset[200]['y'])

tensor(5)


In [ ]:
import torch
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
import torch.nn as nn

class NodeClassificationGNN(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super(NodeClassificationGNN, self).__init__()
        # Graph Convolutional Layers
        self.conv1 = pyg_nn.GCNConv(input_dim, hidden_dim)
        self.bn1 = nn.BatchNorm1d(hidden_dim)
        self.conv2 = pyg_nn.GCNConv(hidden_dim, hidden_dim * 2)
        self.bn2 = nn.BatchNorm1d(hidden_dim * 2)
        self.conv3 = pyg_nn.GCNConv(hidden_dim * 2, hidden_dim * 4)
        self.bn3 = nn.BatchNorm1d(hidden_dim * 4)

        self.fc = nn.Linear(hidden_dim * 4, 3)

        self.dropout = nn.Dropout(0.25)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.bn1(self.conv1(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.bn2(self.conv2(x, edge_index)))
        x = self.dropout(x)
        x = F.relu(self.bn3(self.conv3(x, edge_index)))
        x = self.fc(x)

        return x


In [ ]:
import torch.optim as optim
from torch_geometric.data import DataLoader

train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize model
model = NodeClassificationGNN(input_dim=14, hidden_dim=64)
optimizer = optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.CrossEntropyLoss()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

model.train()
for epoch in range(100):
    total_loss = 0
    for data in train_loader:
        data.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {total_loss/len(train_loader)}')


Epoch 1, Loss: 1.1015046521479643
Epoch 2, Loss: 1.0982651399810095
Epoch 3, Loss: 1.0977923991785248
Epoch 4, Loss: 1.0973961547835336
Epoch 5, Loss: 1.0967042447980393
Epoch 6, Loss: 1.096218670880435
Epoch 7, Loss: 1.0951591696927276
Epoch 8, Loss: 1.094704049922474
Epoch 9, Loss: 1.0942532739617155
Epoch 10, Loss: 1.094014689153419
Epoch 11, Loss: 1.0932715261803057
Epoch 12, Loss: 1.0934144887275135
Epoch 13, Loss: 1.093295978467223
Epoch 14, Loss: 1.0930405613625593
Epoch 15, Loss: 1.09234328797319
Epoch 16, Loss: 1.0926656392039205
Epoch 17, Loss: 1.092713445909062
Epoch 18, Loss: 1.092354872387136
Epoch 19, Loss: 1.0921871924898188
Epoch 20, Loss: 1.092576726775527
Epoch 21, Loss: 1.0922979302675524
Epoch 22, Loss: 1.091967552370188
Epoch 23, Loss: 1.091842048959223
Epoch 24, Loss: 1.0913864271823326
Epoch 25, Loss: 1.0914844306789437
Epoch 26, Loss: 1.0914757744987529
Epoch 27, Loss: 1.091234586355028
Epoch 28, Loss: 1.0917079121404532
Epoch 29, Loss: 1.0913683902544287
Epoch 

In [ ]:
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

def evaluate_accuracy(model, loader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in loader:
            data.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs, 1)
            total += data.y.size(0)
            correct += (predicted == data.y).sum().item()
    accuracy = 100 * correct / total
    return accuracy

test_accuracy = evaluate_accuracy(model, test_loader)
print(f'Test Accuracy: {test_accuracy:.2f}%')


Test Accuracy: 36.78%
